In [54]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [12]:
# read the data set
df=pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [14]:
# Drop the id column due to irrelevance to the modeling task 
df=df.drop(columns='id', axis=1)

In [16]:
#this is how our data set looks like after dropping id column
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [22]:
#assign X(independent) and y(dependent)
X=df.drop(columns='price',axis=1)
y=df[['price']]

In [23]:
#let's see what we got in X
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [24]:
#let's see what we got in y
y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [33]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

print(categorical_cols,'|',numerical_cols)

Index(['cut', 'color', 'clarity'], dtype='object') | Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')


In [48]:
# Define the custom ranking for each ordinal/categorical variable
cut_categories = list(df.cut.value_counts().keys())
color_categories = list(df.color.value_counts().keys())
clarity_categories = list(df.clarity.value_counts().keys())
print(cut_categories)
print(color_categories)
print(clarity_categories)

['Ideal', 'Premium', 'Very Good', 'Good', 'Fair']
['G', 'E', 'F', 'H', 'D', 'I', 'J']
['SI1', 'VS2', 'VS1', 'SI2', 'VVS2', 'VVS1', 'IF', 'I1']


In [49]:
#create pipelines

# Numerical Pipeline
num_pipeline = Pipeline(
                steps = [
                ('imputer',SimpleImputer(strategy='median')),
                ('scaler',StandardScaler())                
                ]
            )

# Categorical Pipeline
cat_pipeline = Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')),
                ('ordinal_encoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
                ('scaler',StandardScaler())
                ]
            )

In [50]:
#creating column transformer 
column_trans = ColumnTransformer(
                [
                ('num_pipeline',num_pipeline,numerical_cols),
                ('cat_pipeline',cat_pipeline,categorical_cols)
                ]
            )

In [55]:
#spliting the data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [56]:
#transform the test data and fit_transform the train data
X_train = pd.DataFrame(column_trans.fit_transform(X_train),columns=column_trans.get_feature_names_out())
X_test = pd.DataFrame(column_trans.transform(X_test),columns=column_trans.get_feature_names_out())

In [57]:
column_trans.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [58]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-1.016395,-0.204317,0.402608,-1.202472,-1.187395,-1.194148,0.132842,-0.668271,-1.088261
1,0.882396,0.720758,-0.118536,0.985177,0.941823,1.036109,0.132842,-0.098646,-0.479273
2,1.529711,0.350728,-1.160823,1.426308,1.394848,1.441611,-0.872563,1.610227,-0.479273
3,1.896523,0.073206,0.923751,1.741402,1.711965,1.702290,0.132842,1.610227,0.738704
4,0.450852,1.738340,1.444895,0.562052,0.525040,0.703019,2.143651,0.470978,-0.479273


In [59]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.628006,0.258221,-0.118536,-0.599293,-0.580341,-0.571414,-0.872563,1.040602,-1.088261
1,2.608569,-2.146973,-0.118536,2.128516,2.201232,1.962969,1.138246,-1.237895,0.738704
2,-1.124281,-1.221899,0.923751,-1.373523,-1.413907,-1.469310,0.132842,-0.668271,1.956680
3,-1.016395,-0.574346,0.923751,-1.157458,-1.160213,-1.194148,0.132842,1.610227,1.956680
4,0.860819,0.628251,-0.639679,0.949167,0.987125,1.007145,-0.872563,0.470978,-1.088261


In [ ]:
#Create an Evaluate Function to use after trainig

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square